In [ ]:
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('apw-notebook.mplstyle')
%matplotlib inline
import sklearn
from astropy.table import Table

# Machine Learning: Introduction

I think of machine learning as a set of algorithms and implementations that make data-driven predictions or decisions. This is why machine learning has become all the buzz in industry, but isn't always applicable in scientific studies; we're often interested in making some inference about a parameter after marginalizing over nuisance parameters so we can learn some thing about "physics." (We're usually not trying to serve ads to teenagers.) Another reason that machine learning algorithms are not obviously applicable to science is that the rarely incorporate observational uncertainties in a justifiable way. More on that later...

The typical problems for which you might consider using machine learning methods can be grouped into four main types of problems:

* **Classification**: I have photometry for 100,000 sources from some imaging survey and I want to know which ones are galaxies, stars, quasars
* **Regression**: I want to learn the parameters of some model, usually so I can make good predictions.
* **Dimensionality reduction**: I have spectra for 100,000 galaxies and I want to decompose the spectra into a mixture of stellar population models.
* **Clustering**: I have colors, sizes, and shapes for 100,000 galaxies and I want to know if there are sub-populations with similar characteristics.

These classes of problems can be further split into two main categories:

* **Supervised learning**: *Classification* and *regression* problems, where you have some set of "features" and some known "labels" for (a subset of) the data. That is, for a subset of the data, we know the true classes or true values. In the first example above, the photometry or colors are the features and the label can be "galaxy", "star", or "quasar". In the case of regression, the labels can be continuous.
* **Unsupervised learning**: *Dimensionality reduction* and *clustering* problems, where you only have a set of "features" and you'd like to find similarities between in order to compress or cluster the data, blindly.

For all of the above, there are many, many algorithms and options within the options. Often, there is no definitive answer or easy way to determine what algorithm will perform best for your use case. It generally takes a lot of trial and error, and help from experts! In some cases, you get the best performance from using *many algorithms simultaneously* and combining the predictions / results (see: Netflix prize).

Some more resources for learning machine learning:
* [Setting the scene](http://scikit-learn.org/stable/tutorial/basic/tutorial.html)
* [Using machine learning with scientific data](http://scikit-learn.org/stable/tutorial/statistical_inference/index.html)
* [Elements of statistical learning, free textbook](https://statweb.stanford.edu/~tibs/ElemStatLearn/)

Luckily, most of the relevant algorithms are implemented in the Python package `scikit-learn` and have a common interface for interacting with them. This makes it fairly easy to write code that is "learning algorithm agnostic," meaning you can swap in and out different methods as you change your mind. The `scikit-learn` developers have also created a rough cheat sheet for algorithms:

In [ ]:
from IPython import display
display.Image('http://scikit-learn.org/stable/_static/ml_map.png')

# Scikit-learn

All of the algorithms in `scikit-learn` have a unified interface for fitting to data and evaluating models. The `scikit-learn` classes expect you to format your data into two numpy arrays: the array of *feature vectors*, `X`, and (if doing supervised learning) the *label vector*, `y`. We'll see examples of how to use these classes below, but here I'll just show an example of how to format your data into these two arrays using one of the demo datasets provided with `scikit-learn`:

In [ ]:
from sklearn.datasets import load_iris

In [ ]:
iris = load_iris()
X = iris.data
y = iris.target

`X` is the feature vector array, a 2D array with shape (number of datapoints, number of features):

In [ ]:
X.shape

So, this dataset contains 150 measurements of 4 features for Irises (flowers): Sepal Length, Sepal Width, Petal Length and Petal Width. Associated with each measurement is a label that designates the type of Iris:

In [ ]:
y.shape, np.unique(y)

There are only 3 unique labels (0, 1, 2), so there are only 3 classes.

### Exercise: read in the table of SDSS photometry and format it into feature and label arrays

The table contains some photometry (g,r,i magnitudes) and other columns -- create a feature array containing the g, r, i magnitudes and the colors (g-r), (r-i), (g-i) (a total of 6 features). Create a label array using the column 'type', which is either 0 (star) or 1 (galaxy).

In [ ]:
filename = '../data/sdss.fit'

In [ ]:
tbl = Table.read(filename)
tbl

In [ ]:
X = np.zeros((len(tbl), 6))

for i,name in enumerate(['g','r','i']):
    X[:,i] = tbl['dered_'+name]
    
X[:,3] = X[:,0]-X[:,1]
X[:,4] = X[:,1]-X[:,2]
X[:,5] = X[:,0]-X[:,2]

y = np.array(tbl['type'])

In [ ]:
X.shape, y.shape

In [ ]:
np.unique(y)

---

# Classification

The goal of a classification problem is to train a model that will successfully predict the classes of new objects that I feed in to the model. Let's stick with the idea of classifying photometric objects as an example. Imagine we have a huge database of photometry for astronomical objects, and for some subset of that database I've gone through by hand and classified the data into two classes: "star" and "galaxy." I don't want to have to go through the whole dataset and label all of the targets, instead I want to train the computer to learn the best way to discriminate these classes based on the smaller *training set* of data that I provide.

A generic problem for any machine learning application is: How do we decide on the model? As mentioned above, that often comes from a mixture of "physical intuition," trial and error, and computational cost. You first have to decide on an algorithm. To do that, you have to think about what accuracy you want, how flexible / nonlinear the model can be (e.g., number of parameters), how much time and data it will take to train the model. Once we choose an algorithm, we then need to figure out a way to determine a "score" for a given choice of parameters. Lastly, we need to decide on how we're going to optimize over the model parameters.

There are many models / algorithms out there. To name a few:
* Nearest neighbors
* Support vector machines
* Decision trees / random forest 
* Neural networks

As a demonstration, here we'll use the K nearest-neighbors algorithm to train a model to predict whether a photometric measurement is a star or a galaxy:

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

Convention is to define the classifier object as "`clf`". For the simplest version of this algorithm, it has 1 tunable parameter: the number of neighbors K to consider.

In [ ]:
clf = KNeighborsClassifier(n_neighbors=5)

Let's start by fitting the classifier to all but the last 128 data points, then predict the classes of the last data points:

In [ ]:
clf.fit(X, y)

In [ ]:
clf.predict(X[:8])

In [ ]:
print("Predicted:", clf.predict(X[:8]))
print("True:     ", y[:8])

Nice, our predictions are pretty good! But how do we evaluate how well our classifier is doing? We need to compute some kind of "score" for our classifier. In machine learning, this is often the most subtle challenge in the methodology. Be aware that scikit-learn often lets you specify or define your own score functions, but for now we're going to table this idea and stick to some simple defaults.

For classification, a common score function is the "misclassification error." This is just the fractional number of data points that we incorrectly classified with whatever classification method we used. We can compute this fairly simply, and then take the compliment of the error to compute the "accuracy" of the classifier:

In [ ]:
misclassification_error = np.sum(clf.predict(X) != y) / len(y)
accuracy = 1 - misclassification_error
accuracy

Scikit-learn also provides a function to compute this:

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
accuracy_score(clf.predict(X), y)

### Exercise: 

How does the accuracy change as you increase the number of neighbors ("K") from 1-10?

In [ ]:
# Solution:
for N in range(1, 11):
    clf = KNeighborsClassifier(n_neighbors=N)
    clf.fit(X, y)
    
    print("n={}, accuracy={:.2f}"
          .format(N, accuracy_score(clf.predict(X), y)))

Do you notice anything strange about the accuracy as we add more neighbors?

---

# Cross-validation

Above, we tried to assess the accuracy of our classifier by computing the misclassification rate. But you may have noticed that the accuracy was best at `k=1`. What's up with that? If we use the same data set to _train_ the classifier and _test_ the classifier, we'll trick ourselves about how well we expect the classifier to perform on new (unclassified) data. In this case, the best accuracy was reached at `k=1` because the nearest neighbor to each predicted data point was the true training point itself!

Cross-validation provides a way to tune or set hyperparameters of machine learning algorithms to try to avoid overfitting to training data. The way these methods generally work for supervised learning problems (like classification) is by training the model on some subset of the full training set, predicting the labels of the held-back data, computing an accuracy metric by comparing to the true labels, then optimizing over the accuracy metric. With jargon, the labeled dataset is typically split into (at least two) subsets called the _training data_ and _test data_. The models are trained on the training data, and then evaluated on the test data, which the model has never seen before. Provided you separate the train/test data in a sensible way, this gives you an unbiased way to validate the models. 

__Exercise__: Split the above datasets into train and test sets. You'll need to split both the features (`X`) and the labels (`y`) in the same way:

In [ ]:
# define the following variables:
# train_X, train_y, test_X, train_y

In [ ]:
# Solution:
idx = np.random.choice(len(X), size=len(X), replace=False)

# 85% training data, 15% test data
split = int(0.85 * len(X))

train_X = X[idx[:split]]
train_y = y[idx[:split]]

test_X = X[idx[split:]]
test_y = y[idx[split:]]

We'll now train the classifier using just the training set, and then evaluate the accuracy on the test set:

In [ ]:
clf.fit(train_X, train_y)

In [ ]:
# Solution:
for N in range(1, 11):
    clf = KNeighborsClassifier(n_neighbors=N)
    clf.fit(train_X, train_y)
    
    print("n={}, accuracy={:.2f}"
          .format(N, accuracy_score(clf.predict(test_X), test_y)))

Let's now use Sciki-learn's cross-validation score function instead, which automatically does the train-test splitting (multiple times). How does the cross-validation score depend on `k`, the number of neighbors?

In [ ]:
from sklearn.model_selection import cross_val_score

In [ ]:
vals = np.arange(2, 100, 5)
mean_scores = []
for n_neighbors in vals:
    clf = KNeighborsClassifier(n_neighbors=n_neighbors)
    scores = cross_val_score(clf, X, y)
    mean_scores.append(np.mean(scores))

In [ ]:
plt.plot(vals, mean_scores)
plt.xlabel('N neighbors')
plt.ylabel('Cross-validation accuracy')

Scikit-learn also has a utility that will automatically do the optimization to find the model parameter that produces the highest-accuracy predictions (see [the documentation for some tips](http://scikit-learn.org/stable/modules/grid_search.html#grid-search-tips)):

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
knn_clf = KNeighborsClassifier()

In [ ]:
train_X = X[:-1024]
train_y = y[:-1024]
test_X = X[-1024:]
test_y = y[-1024:]

In [ ]:
params = {'n_neighbors': np.arange(2, 100, 2)}
cv_clf = GridSearchCV(knn_clf, param_grid=params)
cv_clf.fit(X, y)

In [ ]:
cv_clf.best_score_

### Exercise: 

Repeat the cross-validation above using a Support Vector Machine classifier instead. This classifier has a few parameters, but we'll just tune 1: the parameter `C`. Use `GridSearchCV` to try C = [1,10,100,1000]. What is the best score?

In [ ]:
from sklearn.svm import SVC

In [ ]:
svm_clf = SVC()

In [ ]:
# fill in here